In [92]:
import boto3
from datetime import date
import pandas as pd
from io import StringIO
import streamlit as st

# 오늘 날짜 구성
today = date.today()
year = today.year
month = today.month
day = today.day

# S3 클라이언트 초기화
s3 = boto3.client('s3')

bucket_name = 'pyflink-test-hs'
prefix = f'trade_volume_per_minute/year={year}/month={month:02d}/day={day:02d}/'

@st.cache_data
def load_data(content):
    # df = pd.read_json(content, lines=True)
    df = pd.read_json(StringIO(content), lines=True)

    return df
    
try:
    # 객체 목록 가져오기
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    
    # 객체가 존재하는 경우
    if 'Contents' in response:
        # _SUCCESS 제외 후 가장 최근에 생성된 객체 찾기
        files = [obj for obj in response['Contents'] if not obj['Key'].endswith('_SUCCESS')]
        
        if not files:
            print("파일이 없습니다.")
        else:
            latest_file = max(files, key=lambda x: x['LastModified'])
            latest_key = latest_file['Key']
            # print("가장 최근 파일:", latest_key)

            # 예: 파일 내용 읽기 (텍스트 파일이라고 가정)
            obj = s3.get_object(Bucket=bucket_name, Key=latest_key)
            content = obj['Body'].read().decode('utf-8')
            
            df = load_data(content)
            
            st.title("📊 Trade Volume Per Minute")
            # st.write("Flink에서 저장된 데이터를 Streamlit에서 시각화합니다.")
            
            st.dataframe(df)
    else:
        print("해당 prefix에 객체가 없습니다.")

except Exception as e:
    print("에러 발생:", e)

2025-07-10 11:12:34.626 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-07-10 11:12:34.783 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-10 11:12:34.834 
  command:

    streamlit run /Users/mzc01-hyucksangcho/opt/anaconda3/envs/pyflink_38/lib/python3.8/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-07-10 11:12:34.835 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-10 11:12:34.835 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-10 11:12:34.836 No runtime found, using MemoryCacheStorageManager
2025-07-10 11:12:34.849 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-10 11:12:34.850 Thread 'MainThread': missing ScriptR

      market         window_start           window_end  one_m_trade_volume  \
0    KRW-XRP  2025-07-10 11:10:00  2025-07-10 11:11:00         102495625.0   
1    KRW-TRX  2025-07-10 11:10:00  2025-07-10 11:11:00          15682675.0   
2    KRW-ADA  2025-07-10 11:10:00  2025-07-10 11:11:00          10896300.0   
3  KRW-STEEM  2025-07-10 11:10:00  2025-07-10 11:11:00           9643158.0   
4    KRW-ETH  2025-07-10 11:10:00  2025-07-10 11:11:00           8414575.0   

   row_num  
0        1  
1        2  
2        3  
3        4  
4        5  
